# Acquire

In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from requests import get

In [2]:
response = get('https://www.linkedin.com/jobs/search/?keywords=data%20science')

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')

In [25]:
soup.find_all('div')

[<div class="search-bar__full-placeholder">Data Science in United States</div>,
 <div class="switcher-tabs__trigger-and-tabs"><button aria-expanded="false" class="switcher-tabs__placeholder hide-on-mobile" data-tracking-control-name="public_jobs_switcher-tabs-placeholder"><span class="switcher-tabs__placeholder-text"></span><icon class="switcher-tabs__caret-down-filled onload" data-delayed-url="https://static-exp1.licdn.com/sc/h/7asbl4deqijhoy3z2ivveispv"></icon></button><div class="switcher-tabs hide-on-desktop hide-on-mobile"><ul aria-labelledby="switcher-label" class="switcher-tabs__list"><li class="switcher-tabs__tab switcher-tabs__tab--active"><button class="switcher-tabs__button" data-switcher-type="JOBS" data-tracking-control-name="public_jobs_switcher-tabs-jobs-search-switcher" type="button">Jobs</button></li><li class="switcher-tabs__tab"><button class="switcher-tabs__button" data-switcher-type="PEOPLE" data-tracking-control-name="public_jobs_switcher-tabs-people-search-switch